In [ ]:
import re
import pandas as pd

import gensim
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim.downloader as api
from gensim.parsing.preprocessing import remove_stopwords

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\pokem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
df = pd.read_csv('song_lyrics.csv', nrows=10000)

In [3]:
df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [ ]:
### clean up current lyrics
### remove chorus:, intro:, etc.
### stop words, punctuation

In [94]:
def remove_between_brackets(text):
  """Removes all text between any matching pair of brackets, including the brackets themselves."""
  return re.sub(r'\[.*?\]', '', text)

In [ ]:
def cleanse_lyrics(df):
    sw = stopwords.words('english')

    df['cleaned_lyrics'] = df['lyrics'].apply(remove_between_brackets)
    df['cleaned_lyrics'] = df['cleaned_lyrics'].str.lower()
    df['cleaned_lyrics'] = df['cleaned_lyrics'].apply(remove_stopwords)
    df['tokenized_text'] = df["cleaned_lyrics"].apply(word_tokenize)

    return df

In [104]:
df = cleanse_lyrics(df)
df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language,cleaned_lyrics,tokenized_text
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en,"killa cam, killa cam, cam killa cam, killa cam...","[killa, cam, ,, killa, cam, ,, cam, killa, cam..."
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en,"yeah, hah, yeah, roc-a-fella invite somethin' ...","[yeah, ,, hah, ,, yeah, ,, roc-a-fella, invite..."
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en,maybe cause i'm eatin bastards fiend grub carr...,"[maybe, cause, i, 'm, eatin, bastards, fiend, ..."
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en,"ugh, killa! baby! kanye, 1970s heron flow, huh...","[ugh, ,, killa, !, baby, !, kanye, ,, 1970s, h..."
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en,"ask ""young boy gon' second time around? gon' c...","[ask, ``, young, boy, gon, ', second, time, ar..."


In [67]:
# # clean your sentences
# stopwords = [YOUR_STOPWORDS_HERE]
# cleaned_sentences = []
# for sentence in sentences:
#   cleaned = [word.lower() for word in sentence]
#   cleaned = [word for word in cleaned if word not in stopwords]
#   cleaned_sentences.append(cleaned)

# build a word2vec model on your dataset
sentences = df['tokenized_text'].tolist()
base_model = Word2Vec(vector_size=100, min_count=5)
base_model.build_vocab(sentences)
total_examples = base_model.corpus_count

In [68]:
base_model.train(sentences, total_examples=total_examples, epochs=base_model.epochs) 

(25993762, 35940070)

In [27]:
list(w for w in base_model.wv.index_to_key)[:5]

[',', 'I', 'the', 'you', 'a']

In [69]:
base_model.wv.vectors[:5]

array([[-2.0286001e-01,  4.3490484e-01, -3.6596939e-01, -9.9343044e-01,
         1.2629777e-01, -5.4810727e-01, -6.8511891e-01,  6.1739576e-01,
         6.6720665e-02,  1.0636948e+00, -1.7869288e-01,  7.7993184e-01,
        -1.7503660e-02, -3.0275127e-01,  9.7214353e-01,  5.1051933e-01,
         3.2082838e-01,  1.0667556e-01,  5.7888609e-01,  3.2736555e-01,
        -2.5839835e-01,  2.2825576e-01, -5.6971192e-01, -7.1697488e-02,
        -1.2395582e+00, -1.1794237e+00,  1.9163245e-01, -5.7696766e-01,
        -8.6238414e-01, -8.0855572e-01, -1.5279549e+00, -6.7870569e-01,
        -9.6184969e-01, -8.0445427e-01, -3.8958216e-01,  3.4506679e-01,
         4.0108714e-02,  4.6323735e-01,  5.2820307e-01,  5.2368408e-01,
         1.1165643e+00,  1.0798833e+00, -3.0083838e-01,  8.4686438e-03,
         6.3838387e-01, -7.2977841e-02, -3.9434960e-01,  5.0134283e-01,
        -7.8882903e-01, -4.6081910e-01, -2.8921324e-01,  1.4373095e+00,
         1.4088441e+00,  4.0768978e-01,  2.7414035e-02,  3.67341

In [ ]:
def apply_word2vec(sentences):
  """
  apply_word2vec
  params: sentences -> 'tokenized_text'
  returns: word2vec model
  
  Access vectors from base_model.wv.vectors and base_model.wv.index_to_key
  """
  base_model = Word2Vec(vector_size=100, min_count=5)
  base_model.build_vocab(sentences)
  # base_model.train(sentences, total_examples=base_model.corpus_count, epochs=base_model.epochs) 
  return base_model

In [34]:
model = apply_word2vec(df)

### Embedding: GloVe

In [ ]:
def apply_glove(sentences, model="glove-wiki-gigaword-100"):

    print("Models available for use:")
    print(list(gensim.downloader.info()['models'].keys()))

    glove_model = api.load(model)

    ### initialize model
    base_model = Word2Vec(vector_size=100, min_count=1)
    base_model.build_vocab(sentences)
    total_examples = base_model.corpus_count

    base_model.build_vocab(glove_model.index_to_key, update=True)
    base_model.train(sentences, total_examples=total_examples, epochs=base_model.epochs)

    return base_model

In [112]:
glove_model = api.load("glove-wiki-gigaword-100")

In [89]:
apply_glove()

Models available for use:
['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
#### build word embedding matrix for GloVe

### Embedding: BERT and DistilBERT

In [ ]:
#import library
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import numpy as np

#load DistilBERT tokenizer and a pretrained model to avoid training from scratch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
#tokenize and obtain embeddings
def get_lyrics_embedding(lyrics):
    tokens = tokenizer(
        lyrics,
        truncation = True,
        padding = True, 
        max_length = 512, #DistilBERT has a max token limit of 512
        return_tensors = "pt" #converting output as PyTorch since DistilBERT expects tensors not token IDs
        )
    
    with torch.no_grad(): #removes gradient calculation to save memory usage since inferences are not needed as we are predicting, not training
        output = model(**tokens)

    cls_embedding = output.last_hidden_state[:, 0] #extract first token with CLS
    cls_embedding = cls_embedding.detach() #detach from PyTorch's gradient computation 
    cls_embedding = cls_embedding.cpu() #converting tensor to CPU to ensure compatability (ie. NumPy array conversion)
    cls_embedding = cls_embedding.squeeze() #remove any extra dimensions
    
    embedding = cls_embedding.numpy() #converting into NumPy array
    return embedding

#converting lyrics into embeddings using nrows
embeddings = np.array([get_lyrics_embedding(lyric) for lyric in df['lyrics']])


### Bag of Words Model

### TF-IDF

### Question: Are we using Keras / PyTorch?
This may change the format and implementation of the current method of embedding.